# Lists changes in issues linked to specified epics

Specify epics

In [ ]:
epic = ['VXT-3080', 'VXT-3952']

In [ ]:
import pandas as pd
from jira import JIRA

#store credentials in ~/.rcnet file
jira = JIRA('https://kainos-evolve.atlassian.net')
jql = '"Epic Link" in (' + ", ".join(epic) + ')'

print(jql)
epicIssues = jira.search_issues(jql)

In [ ]:
from pandas.tseries.offsets import BDay

changes = pd.DataFrame()
changes['epic'] = ''
changes['key'] = ''
changes['summary'] = ''
changes['updateDate'] = ''
changes['field'] = ''
changes['oldValue'] = ''
changes['newValue'] = ''
changes['author'] = ''

for issue in epicIssues:
    issue = jira.issue(issue.key, expand='changelog')
    changelog = issue.changelog
    for history in changelog.histories:
        for item in history.items:
            try:
                author = history.author.name
            except AttributeError:
                author = 'NotFound'
                
            changes = changes.append(
                {'key': issue.key,
                 'updateDate': history.created,
                 'author': author,
                 'field': item.field,
                 'oldValue': item.fromString,
                 'newValue': item.toString,
                 'summary': issue.fields.summary,
                 'epic': issue.fields.customfield_10008
                 }, ignore_index=True)


#sort values
changes.sort_values(["key", "updateDate",], inplace=True)

#show only items from last 5 business days
changes = changes.loc[(pd.to_datetime(changes['updateDate']) > (pd.to_datetime('today') - BDay(5)))]

#don't show changes in following fields
changes = changes.loc[~(changes['field'].isin(['status', 'assignee', 'Sprint', 'reporter']))]
changes